# Download Dataset of Google

Download data from below link with mark and area and label
```
https://storage.googleapis.com/openimages/web/visualizer/index.html?set=train&type=detection&c=%2Fm%2F01mqdt
```

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### download model

In [2]:
# clone darknet repo
# % cd ../gdrive/MyDrive/Yolo/yolov3
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15363, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 15363 (delta 0), reused 3 (delta 0), pack-reused 15358
Receiving objects: 100% (15363/15363), 14.00 MiB | 17.51 MiB/s, done.
Resolving deltas: 100% (10328/10328), done.


### download data

In [ ]:
! git clone https://github.com/EscVM/OIDv4_ToolKit.git
! mkdir dataset/google
% cd OIDv4_ToolKit
! pip3 install -r requirements.txt
! python3 main.py downloader --classes Traffic_sign --type_csv validation --Dataset ../../dataset/google/ -y
! python3 main.py downloader --classes Traffic_sign --type_csv test  --Dataset ../../dataset/google/ -y
! python3 main.py downloader --classes Traffic_sign --type_csv train --Dataset ../../dataset/google/ -y

Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 422, done.
remote: Total 422 (delta 0), reused 0 (delta 0), pack-reused 422
Receiving objects: 100% (422/422), 34.08 MiB | 19.55 MiB/s, done.
Resolving deltas: 100% (146/146), done.
mkdir: cannot create directory ‘dataset/google’: No such file or directory
/content/OIDv4_ToolKit
     |████████████████████████████████| 3.8 MB 4.3 MB/s 
     |████████████████████████████████| 547 kB 51.0 MB/s 
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 8.1 MB 48.8 MB/s 
     |████████████████████████████████| 138 kB 42.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18
    Uninstalling docutils-0.18:
      Successfully uninstalled docutils-0.18
ERROR: pip's dependency resolver does not currently take

### organize the data

In [ ]:
% cd ../dataset
! mkdir obj
! cp google/test/Traffic\ sign/Label/* obj
! cp google/test/Traffic\ sign/*.jpg obj
! cp google/train/Traffic\ sign/Label/* obj
! cp google/train/Traffic\ sign/*.jpg obj
! cp google/validation/Traffic\ sign/Label/* obj
! cp google/validation/Traffic\ sign/*.jpg obj

/content/dataset


In [ ]:
!sed -i 's/Traffic sign/0/g' obj/*.txt

In [ ]:
! pwd

/content/dataset


### change annotation

In [ ]:
import os 
filelist = os.listdir('obj')
textfilelist = list(filter(lambda x : '.txt' in x, filelist))

In [ ]:
import cv2
for textfile in textfilelist :
  file = open("obj/"+textfile,mode='r+')
  all_of_it = file.read()
  lines = all_of_it.split('\n')[:-1]  
  file.close()
  
  im = cv2.imread('obj/'+textfile.split('.txt')[0]+'.jpg')
  h, w, c = im.shape
  print(h, w, c)

  new_file = open("obj/"+textfile, "w+")
  for line in lines :
    print(line)
    content = line.split()
    print(content)
    min_x = float(content[1]) 
    min_y = float(content[2])
    max_x = float(content[3])
    max_y = float(content[4])
    content[1] = str(float(min_x/w))
    content[2] = str(float(min_y/h))
    content[3] = str(float(max_x/w))
    content[4] = str(float(max_y/h))
    print(content)
    new_line = " ".join(content)+"\n"
    print(new_line)
    new_file.write(new_line)
  new_file.close()
  # break

Streaming output truncated to the last 5000 lines.
0 981.12 213.12 1023.36 268.160256
['0', '981.12', '213.12', '1023.36', '268.160256']
['0', '0.958125', '0.2775', '0.999375', '0.349167']
0 0.958125 0.2775 0.999375 0.349167

1024 765 3
0 237.00006000000002 252.99967999999998 523.999755 632.999936
['0', '237.00006000000002', '252.99967999999998', '523.999755', '632.999936']
['0', '0.309804', '0.24706999999999998', '0.6849670000000001', '0.618164']
0 0.309804 0.24706999999999998 0.6849670000000001 0.618164

768 1024 3
0 284.000256 212.000256 894.0001279999999 481.99987200000004
['0', '284.000256', '212.000256', '894.0001279999999', '481.99987200000004']
['0', '0.277344', '0.276042', '0.8730469999999999', '0.627604']
0 0.277344 0.276042 0.8730469999999999 0.627604

768 1024 3
0 65.84115200000001 240.97536 173.26592 419.53919999999994
['0', '65.84115200000001', '240.97536', '173.26592', '419.53919999999994']
['0', '0.06429800000000001', '0.31377', '0.169205', '0.546275']
0 0.0642980000000

In [ ]:
!cp -r obj ../darknet/data

### write obj.names

In [ ]:
!echo "Traffic sign" > ../darknet/data/obj.names

### write obj.data


In [ ]:
!echo -e """classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/gdrive/MyDrive/Yolo/yolov3""" > ../darknet/data/obj.data

### make train.txt


In [ ]:
% pwd
% cd ..

/content


In [ ]:
import os
image_files = []
os.chdir(os.path.join("dataset", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
% pwd
with open("../darknet/data/train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")

### make cfg file

In [ ]:
% cd darknet
! cp cfg/yolov3.cfg cfg/yolov3_traffic_signal.cfg
!sed -i 's/batch=1/batch=8/' cfg/yolov3_traffic_signal.cfg
!sed -i 's/subdivisions=1/subdivisions=1/' cfg/yolov3_traffic_signal.cfg
!sed -i 's/max_batches = 500200/max_batches = 2200/' cfg/yolov3_traffic_signal.cfg
!sed -i 's/steps=400000,450000/steps=1600,1800/' cfg/yolov3_traffic_signal.cfg
!sed -i 's/filters=255/filters=18/g' cfg/yolov3_traffic_signal.cfg
!sed -i 's/classes=80/classes=1/g' cfg/yolov3_traffic_signal.cfg

/content/darknet


### build darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

# verify CUDA
!/usr/local/cuda/bin/nvcc --version

# make darknet (build)
!make

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int,

# Download Pre-trained Weight

In [ ]:
# get yolov3 pretrained coco dataset weights
!wget https://pjreddie.com/media/files/yolov3.weights

--2021-11-16 19:29:46--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  23.5MB/s    in 11s     

2021-11-16 19:29:57 (21.7 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
# upload pretrained convolutional layer weights
!wget http://pjreddie.com/media/files/darknet53.conv.74

URL transformed to HTTPS due to an HSTS policy
--2021-11-16 19:29:57--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  23.5MB/s    in 7.4s    

2021-11-16 19:30:05 (21.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

# training with my data

In [ ]:
# train your custom detector
!./darknet detector train data/obj.data cfg/yolov3_traffic_signal.cfg darknet53.conv.74 -dont_show